<a href="https://colab.research.google.com/github/TDStriker/Projects-in-ML/blob/main/ML_Proj_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1: For this project I would like to make a model that can predict if a student will pass their class. Logistic regression should work well for this problem since itis taking information that can be expressed numerically and using it to perform a binary classification task

Task 2: For the data analysis the data was split into features and grades so that grades could be used as the target value. The first and second period grades were removed so as to focus specifically on the final grade prediction.

There is also data removed later on as it was found to be statistically insignificant in its p-score evaluation

During data processing data is split between training and testing data

In [851]:
pip install ucimlrepo

In [852]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from mlxtend.preprocessing import TransactionEncoder
import seaborn as sns
from scipy.stats import pearsonr


In [853]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
student_performance = fetch_ucirepo(id=320)

# data (as pandas dataframes)
feats = student_performance.data.features
grades = student_performance.data.targets
size = len(feats)

# metadata
print(student_performance.metadata)

# variable information
print(student_performance.variables)


{'uci_id': 320, 'name': 'Student Performance', 'repository_url': 'https://archive.ics.uci.edu/dataset/320/student+performance', 'data_url': 'https://archive.ics.uci.edu/static/public/320/data.csv', 'abstract': 'Predict student performance in secondary education (high school). ', 'area': 'Social Science', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 649, 'num_features': 30, 'feature_types': ['Integer'], 'demographics': ['Sex', 'Age', 'Other', 'Education Level', 'Occupation'], 'target_col': ['G1', 'G2', 'G3'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Fri Jan 05 2024', 'dataset_doi': '10.24432/C5TG7T', 'creators': ['Paulo Cortez'], 'intro_paper': {'title': 'Using data mining to predict secondary school student performance', 'authors': 'P. Cortez, A. M. G. Silva', 'published_in': 'Proceedings of 5th Annual Future Business Technology Conference', 'year'

In [854]:
print(feats)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
644     MS   F   19       R     GT3       T     2     3  services     other   
645     MS   F   18       U     LE3       T     3     1   teacher  services   
646     MS   F   18       U     GT3       T     1     1     other     other   
647     MS   M   17       U     LE3       T     3     1  services  services   
648     MS   M   18       R     LE3       T     3     2  services     other   

     ... higher internet  romantic  famrel  freetim

In [855]:
print(grades)

     G1  G2  G3
0     0  11  11
1     9  11  11
2    12  13  12
3    14  14  14
4    11  13  13
..   ..  ..  ..
644  10  11  10
645  15  15  16
646  11  12   9
647  10  10  10
648  10  11  11

[649 rows x 3 columns]


Data Statistics

Drop Insignificant Data.
Any data with a p-stat > .05 is dropped.

In [856]:
'''
P-Value failures:
  famsize: .25
  Pstatus: .9847
  Fjob: .1778
  Schoolsup: .09
  Famsup: .13
  Paid: .16
  Activities: .12
  Nursery: .465
  Romantic: .21
  Family Relationships: .1
  Go out: .0255
  Health: .012
  Absences: .0199
'''

'\nP-Value failures:\n  famsize: .25\n  Pstatus: .9847\n  Fjob: .1778\n  Schoolsup: .09\n  Famsup: .13\n  Paid: .16\n  Activities: .12\n  Nursery: .465\n  Romantic: .21\n  Family Relationships: .1\n  Go out: .0255\n  Health: .012\n  Absences: .0199\n'

In [857]:
feats = feats.drop(['famsize','Pstatus','Fjob','schoolsup','famsup','paid','activities','nursery','romantic','famrel','goout','health','absences'],axis=1)

Convert categorical data into quantitative data

In [858]:
#Convert all binary categories to binary values
le = LabelEncoder()
feats['sex'] = le.fit_transform(feats['sex']) #female should be 0
print("Female:",le.transform(['F']))
feats['address'] = le.fit_transform(feats['address']) #urban should be 0
print("Urban:",le.transform(['U']))
feats['school'] = le.fit_transform(feats['school'])
feats['higher'] = le.fit_transform(feats['higher']) #yes should be 0
print("Plan for higher education:",le.transform(['yes']))
feats['internet'] = le.fit_transform(feats['internet']) #no should be 0
print("Internet:",le.transform(['yes']))


Female: [0]
Urban: [1]
Plan for higher education: [1]
Internet: [1]


In [859]:
feats['Mjob'] = le.fit_transform(feats['Mjob']) #home = 0, health = 1, other = 2, services = 3, teacher = 4
print("Home,Health,Other,Services,Teacher:",le.transform(['at_home','health','other','services','teacher']))
feats['reason'] = le.fit_transform(feats['reason']) #course = 0, other = 2, home = 1
print("Course,Other,Home:",le.transform(['course','other','home']))
feats['guardian'] = le.fit_transform(feats['guardian']) #mother = 1, father = 0, other = 2
print("Mother,Father,Other:",le.transform(['mother','father','other']))

Home,Health,Other,Services,Teacher: [0 1 2 3 4]
Course,Other,Home: [0 2 1]
Mother,Father,Other: [1 0 2]


In [860]:
print(feats)

     school  sex  age  address  Medu  Fedu  Mjob  reason  guardian  \
0         0    0   18        1     4     4     0       0         1   
1         0    0   17        1     1     1     0       0         0   
2         0    0   15        1     1     1     0       2         1   
3         0    0   15        1     4     2     1       1         1   
4         0    0   16        1     3     3     2       1         0   
..      ...  ...  ...      ...   ...   ...   ...     ...       ...   
644       1    0   19        0     2     3     3       0         1   
645       1    0   18        1     3     1     4       0         1   
646       1    0   18        1     1     1     2       0         1   
647       1    1   17        1     3     1     3       0         1   
648       1    1   18        0     3     2     3       0         1   

     traveltime  studytime  failures  higher  internet  freetime  Dalc  Walc  
0             2          2         0       1         0         3     1     1  
1

In [861]:
#Merge data to do a training/testing split
merged = pd.merge(feats,grades['G3'],left_index=True,right_index=True)
#Shuffle
merged = merged.sample(frac = 1)
part = int(size*9/10)
test = merged.iloc[part:]
train = merged.iloc[:part]

In [862]:
#Separate into features and grades
train_grade = train["G3"]
train_feat = train.drop(columns=["G3"])
test_grade = test["G3"]
test_feat = test.drop(columns=["G3"])

In [863]:
#Should make all grades > 12 equal to 1, all less equal to 0
#Boolean list of if they passed
passed = train_grade.where(train_grade>12,0)
passed = passed.where(passed<=12,1)

In [864]:
#How many passed vs failed in the training data
passed.value_counts()

0    335
1    249
Name: G3, dtype: int64

Data Visualization

In [865]:
#Correlation coefficients
merged.corr()

,school,sex,age,address,Medu,Fedu,Mjob,reason,guardian,traveltime,studytime,failures,higher,internet,freetime,Dalc,Walc,G3
school,1.000000,-0.083050,0.087170,-0.354520,-0.254787,-0.209806,-0.206829,-0.109754,-0.062333,0.252936,-0.137857,0.113788,-0.136112,-0.240486,0.034666,0.047169,0.014169,-0.284294
sex,-0.083050,1.000000,-0.043662,0.025503,0.119127,0.083913,0.149635,0.010732,-0.036811,0.040880,-0.206214,0.073888,-0.058134,0.065911,0.146305,0.282696,0.320785,-0.129077
age,0.087170,-0.043662,1.000000,-0.025848,-0.107832,-0.121050,-0.071770,-0.025855,0.266830,0.034490,-0.008415,0.319968,-0.265497,0.013115,-0.004910,0.134768,0.086357,-0.106505
address,-0.354520,0.025503,-0.025848,1.000000,0.190320,0.141493,0.159761,-0.002367,-0.019359,-0.344902,0.062023,-0.063824,0.076706,0.175794,-0.036647,-0.047304,-0.012416,0.167637
Medu,-0.254787,0.119127,-0.107832,0.190320,1.000000,0.647477,0.459337,0.132855,-0.014044,-0.265079,0.097006,-0.172210,0.213896,0.266052,-0.019686,-0.007018,-0.019766,0.240151
Fedu,-0.209806,0.083913,-0.121050,0.141493,0.647477,1.000000,0.290703,0.080760,-0.101764,-0.208288,0.050400,-0.165915,0.191735,0.183483,0.006841,0.000061,0.038445,0.211800
Mjob,-0.206829,0.149635,-0.071770,0.159761,0.459337,0.290703,1.000000,0.059397,0.008196,-0.164126,0.057176,-0.117882,0.148116,0.260658,0.053927,0.049576,0.025657,0.148252
reason,-0.109754,0.010732,-0.025855,-0.002367,0.132855,0.080760,0.059397,1.000000,-0.065834,-0.092522,0.135874,-0.144459,0.091324,0.110168,-0.047001,-0.010735,0.010612,0.124969
guardian,-0.062333,-0.036811,0.266830,-0.019359,-0.014044,-0.101764,0.008196,-0.065834,1.000000,0.026519,-0.009911,0.169605,-0.114735,-0.000412,0.051442,0.023330,-0.008312,-0.079609
traveltime,0.252936,0.040880,0.034490,-0.344902,-0.265079,-0.208288,-0.164126,-0.092522,0.026519,1.000000,-0.063154,0.097730,-0.071958,-0.190826,0.000937,0.092824,0.057007,-0.127173


Task 3:

In [866]:
#Hyperparameters
learning_rate = 1e-2
num_epochs = 10

In [867]:
def sigmoid(a):
  return 1/(1+np.e**(-a))

In [868]:
def update(w,X,Y,b):
  A = sigmoid(np.dot(X,np.transpose(w)) + b)

  n = len(X)
  cost=-1/n * np.sum(Y * np.log(A) + (1-Y) * (np.log(1-A)))

  dw = np.dot((A-Y).T,X)/n
  db= np.sum(A-Y)/n

  w = w - learning_rate*dw
  b = b - learning_rate*db
  return (w,b)

In [869]:
def test(w,b):
  ind = 0
  correct = 0
  for _, row in test_feat.iterrows():
    pred = 1 if sigmoid(np.dot(np.transpose(w),row) + b) > 0.5 else 0
    act = 1 if test_grade.iloc[ind] > 12 else 0
    print("Predicted:",pred)
    print("Actual:",act)
    if(pred == act):
      correct+=1
    ind+=1
  print('Test Accuracy: {0:0.2f}%'.format(100*(correct/len(test_grade))))

Batch Gradient Descent

In [870]:
b = 0
w = [0] * train_feat.shape[1]

for i in range(num_epochs):
  w,b=update(w,train_feat,passed,b)

In [871]:
test(w,b)

Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 

Stochastic Gradient Descent

In [872]:
b = 0
w = [0] * train_feat.shape[1]
ind = 0
for i in range(num_epochs):
  ind = 0
  ordering = train.sample(frac=1)
  rand_grade = ordering['G3']
  rand_feat = ordering.drop(['G3'],axis=1)
  for _, row in rand_feat.iterrows():
    y = 1 if rand_grade.iloc[ind] > 12 else 0
    w,b=update(w,row,y,b)
    ind+=1

In [873]:
test(w,b)

Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 1
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 1
Predicted: 1
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 0
Actual: 0
Predicted: 1
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 1
Actual: 0
Predicted: 1
Actual: 0
Predicted: 1
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 0
Predicted: 0
Actual: 1
Predicted: 1
Actual: 1
Predicted: 1
Actual: 1
Predicted: 

Mini-Batch

In [874]:
b = 0
w = [0] * train_feat.shape[1]
batches=10
batch_size = int(len(train_feat)/batches)

mergbat = pd.merge(train_feat,passed,left_index=True,right_index=True)

for i in range(num_epochs):
  ordering = mergbat.sample(frac=1)
  chunks = [ordering[j:j+batch_size] for j in range(0,len(ordering),batch_size)]

  for k in range(batches):
    batch = chunks[k]
    rand_grade = batch['G3']
    rand_feat = batch.drop(['G3'],axis=1)

    w,b=update(w,rand_feat,rand_grade,b)

In [875]:
test(w,b)

Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 1
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 1
Predicted: 1
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 1
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 1
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 

Task 4:

The main evaluation metric I will be using is a comparison of accuracies

Momentum

In [876]:
gamma = 0.9
def momentum(w,X,Y,b,pdw,pdb):
  A = sigmoid(np.dot(X,np.transpose(w)) + b)

  n = len(X)
  cost=-1/n * np.sum(Y * np.log(A) + (1-Y) * (np.log(1-A)))

  dw = np.dot((A-Y).T,X)/n
  db= np.sum(A-Y)/n

  #Change in weights and bias
  cw = learning_rate*dw + np.dot(gamma,pdw)
  cb = learning_rate*db + np.dot(gamma,pdb)

  w = w - cw
  b = b - cb
  return (w,b,cw,cb)

In [877]:
b = 0
w = [0] * train_feat.shape[1]
cb = 0
cw = [0] * train_feat.shape[1]
ind = 0
for i in range(num_epochs):
  ind = 0
  ordering = train.sample(frac=1)
  rand_grade = ordering['G3']
  rand_feat = ordering.drop(['G3'],axis=1)
  for _, row in rand_feat.iterrows():
    y = 1 if rand_grade.iloc[ind] > 12 else 0
    w,b,cw,cb=momentum(w,row,y,b,cw,cb)
    ind+=1

In [878]:
test(w,b)

Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 1
Predicted: 0
Actual: 1
Predicted: 1
Actual: 0
Predicted: 1
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 1
Predicted: 1
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 0
Predicted: 1
Actual: 0
Predicted: 1
Actual: 0
Predicted: 1
Actual: 1
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 0
Predicted: 1
Actual: 0
Predicted: 1
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 1
Actual: 1
Predicted: 1
Actual: 0
Predicted: 0
Actual: 1
Predicted: 1
Actual: 1
Predicted: 1
Actual: 1
Predicted: 

Adagrad

In [879]:
epsilon = 1e-8
def adagrad(w,X,Y,b,Gw,Gb):
  A = sigmoid(np.dot(X,np.transpose(w)) + b)

  n = len(X)
  cost=-1/n * np.sum(Y * np.log(A) + (1-Y) * (np.log(1-A)))

  dw = np.dot((A-Y).T,X)/n
  db= np.sum(A-Y)/n

  Gw+=np.dot(dw,dw)
  Gb+=np.dot(db,db)

  lrw = learning_rate/np.sqrt(Gw+epsilon)
  lrb = learning_rate/np.sqrt(Gb+epsilon)

  w = w - lrw*dw
  b = b - lrb*db
  return (w,b,Gw,Gb)

In [880]:
Gw = [0] * train_feat.shape[1]
Gb = 0
b = 0
w = [0] * train_feat.shape[1]
cb = 0
cw = [0] * train_feat.shape[1]
ind = 0
for i in range(num_epochs):
  ind = 0
  ordering = train.sample(frac=1)
  rand_grade = ordering['G3']
  rand_feat = ordering.drop(['G3'],axis=1)
  for _, row in rand_feat.iterrows():
    y = 1 if rand_grade.iloc[ind] > 12 else 0
    w,b,Gw,Gb=adagrad(w,row,y,b,Gw,Gb)
    ind+=1

In [881]:
test(w,b)

Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 0
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 0
Actual: 1
Predicted: 

The optimization algorithms seem to get a similar result to the standard gradient descent algorithms. Due to the randomness of the data used in training, there are fluctuations in the resulting accuracies. On average the optimization algorithms seem to be less stable in these fluctuations and I have seen then get lower accuracies, however that may just be a consequence of the given samples.

Explanation of Results:

For many of the runs the models ends up getting around 50% accuracy and predicts all or mostly the same result for every data point. This is likely because in those training slices one of the results ended up being too predominant making the model simply learn to pick that answer.

Across the models it usually gets anywhere from 50% to 70% accuracy. This is likely since the data is usable to help predict the outcome, but the reality of a student's grades depend on far more than the traits provided in the data. When dealing with information on people there are unreportable factors that many skew the results since no two people are the same. Many of the categories in the data also seem to rely on self-reporting and self-evaluation, which may also lead to innaccuracies in the dataset.